In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pylandau

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run

0) Event
1) Pixel Column
2) Row
3) Baseline Est.
4) Amplitude Est.
5) Noise RMS Est.
6) Baseline
7) Amplitude
8) Risetime
9) ToA
10) ToT
11) chi2red
12) TLU time stamp
13) fit param 0
14) fit param 1

In [ ]:
data = {}

if True:
    runIDs = arr([184,190,192])
    for runID in runIDs:
        data[runID] = load_run(runID, 200000*4)
else:
    runID = 192
    # run[runID] = load_run(runID)
    data[runID] = load_run(runID, 2000000*4, "hits")

In [ ]:
runID = 184

In [ ]:
fig, ax = Hist.create_fig()

binN = 40
binRange = [0,10000]

# mask_aboveThreshold = np.where(run[runID]["data"][:,:,7] > 0, True, False)

mask_aboveThreshold = None

dataString = data[runID]["M"]["ToA"]
hist, bins = np.histogram(data[runID]["data"][dataString], bins=binN, range=binRange)

Hist.draw(ax[0], hist, bins)
# Hist.draw(run[runID], fig, ax, 9, binN=40, binRange=binRange, mask=mask_aboveThreshold, color="black", label="A > thr")
# scale = 1000
# for i in range(4):
#     ticks = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x*scale))
#     ax[i].xaxis.set_major_formatter(ticks)

Hist.finalize(data[runID], fig, ax, "ToA [ns]", "Entries", "Time of Arrival", logy=False)
fig.savefig("ToA.pdf")

In [ ]:
import matplotlib.ticker as ticker

if runID == 192:
    binRange = [-.0001,0]
else:
    binRange = [-.0015,0]
# binRange = None
limits = arr([0.022,0.05,0.1,0.15,0.27,0.4])

fig, ax = Hist.create_fig()

masks = Hist.createMask(run[runID], 4, limits)
labels = ["{:.3f} < A < {:.3f}".format(limits[i], limits[i+1]) for i in range(len(limits)-1)]

# Hist.drawStacked(run[runID], fig, ax, 14, binN=50, binRange=binRange, masks=masks, colors=None, labels=labels)
Hist.drawMultiple(run[runID], fig, ax, 14, binN=50, binRange=binRange, masks=masks, colors=None, labels=labels)

scale = 1000
for i in range(4):
    ticks = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x*scale))
    ax[i].xaxis.set_major_formatter(ticks)

Hist.finalize_fig(run[runID], fig, ax, "Falling Edge Sope [mV/ns]", "Entries", "Falling Edge Slope", logy=False)

In [ ]:
binRange = [-0.075,0.3]

fig, ax = Hist.create_fig()

mask_aboveThreshold = np.where(run[runID]["data"][:,:,7] > 0, True, False)
mask_aboveThreshold_noSharing = np.zeros(run[runID]["data"][:,:,4].shape, dtype=bool)
for i in range(len(mask_aboveThreshold_noSharing)): # loop through events
    mask_aboveThreshold_noSharing[i] = np.where(run[runID]["data"][i,:,4] == np.max(run[runID]["data"][i,:,4]),
        True, False)
# Hist.draw(run[runID], fig, ax, 4, binN=200, binRange=binRange, mask=None, color="black", label="all")
Hist.draw(run[runID], fig, ax, 4, binN=100, binRange=binRange, mask=mask_aboveThreshold, color="C0", label="> thr")
# Hist.draw(run[runID], fig, ax, 4, binN=100, binRange=binRange, mask=mask_aboveThreshold_noSharing, color="C1", label="> thr, no C sharing")
# for i in range(4):
#     ax[i].set_ylim(0,1200)


Hist.finalize_fig(run[runID], fig, ax, xlabel="Amplitude [V]", ylabel="Counts", title="Amplitude", logy = True)

for i in range(4):
    data = run[runID]["data"][:,i,4]
    data = np.sort(data)
    percentile = .99
    position = int(len(data)*percentile)
    ax[i].axvline(data[position], color="red", linestyle="--", label="exclude {:.2f}%,\nA={:.3f}mV".format(percentile*100,data[position]*1000))
    ax[i].legend(facecolor='white', framealpha=1)

In [ ]:
binRange = None
fig, ax = Hist.create_fig()

Hist.draw(run[runID], fig, ax, 4, binN=100, binRange=binRange, mask=None, color="black", label="all")


In [ ]:
binRange = None
binN = 50
fig, ax = Hist.create_fig()

Hist.draw(run[runID], fig, ax, 5, binN=binN, binRange=binRange, mask=None, color="black", label="all")

fig.tight_layout()
Hist.finalize_fig(run[runID], fig, ax, xlabel="Baseline [V]", ylabel="Counts", title="Baseline RMS", logy = False)

In [ ]:
binRange = [-.045,.03]
binN = 50
fig, ax = Hist.create_fig()

# masks = Hist.createMask(run[runID], 3, limits)
Hist.draw(run[runID], fig, ax, 3, binN=binN, binRange=binRange, mask=None, color="black", label="all")

fig.tight_layout()
Hist.finalize_fig(run[runID], fig, ax, xlabel="Baseline RMS [V]", ylabel="Counts", title="Baseline", logy = False)

In [ ]:
i_pix = 0
plt.plot(np.arange(len(run[runID]["data"][:,i_pix,12])), run[runID]["data"][:,i_pix,12])

In [ ]:
ar1 = np.arange(0,16)
ar2 = np.arange(0,16) + 10
ar = arr([ar1,ar2])
splitLen = 3
splitN = np.ma.size(ar, axis=1)//splitLen
splitRes = np.ma.size(ar, axis=1)%splitLen
print("step-len", splitLen, ", splits", splitN, ", rest", splitRes)
if splitRes > 0:
    ar = ar[:,:-splitRes]
print("before:\n", ar)

ar = ar.reshape(2,splitN,splitLen)
print("step 1:\n", ar)

ar = np.mean(ar, axis=2)
print("step 2:\n", ar)


ar = np.mean(ar.reshape(2,splitN,splitLen), axis=2)

# temp =  np.zeros([2,splitN,splitLen])
# for i in range(2):
#     for j in range(splitN):
#         temp[i,j] = ar[i,j*splitLen:(j+1)*splitLen]
# print(temp)




In [ ]:
def create_fig(figsize=(8,6)):
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.tick_params("both", direction="in", top=True, right=True)
    return fig, [ax]


def draw_scatter(single_run, fig, ax, dataColumnX, dataColumnY, mask=None, color="black", label=None, averageN=1):
    
    data = arr([single_run["data"][:,0,dataColumnX], single_run["data"][:,0,dataColumnY]])
    
    if mask is None:
        mask = np.ones(data[0].shape, dtype=bool)
    data = data[:,mask]
    
    # data[0] = data[0] * 1E-9 / 3600
    data[1] = data[1] * 1E3
    
    if averageN:
        if len(data[0]) != len(data[1]):
            print("Warning: data columns have different lengths:", len(data[0]), len(data[1]), " Averaging went wrong.")
        
        splitLen = averageN # length of one fragment
        splitN = np.ma.size(data, axis=1)//splitLen # number of fragments the array is split into
        splitRes = np.ma.size(data, axis=1)%splitLen # number of leftover data points
        print("splitting array of length", np.ma.size(data, axis=1), "into", splitN, "parts of length", splitLen, "and", splitRes, "is leftover.")

        if splitRes > 0:
            data = data[:,:-splitRes]
            
        data = data.reshape(2,splitN,splitLen)
        data_unc = np.std(data, axis=2)
        
        data = np.mean(data, axis=2)
                
        if len(data[0]) != len(data[1]):
            print("Warning: data columns have different lengths:", len(data[0]), len(data[1]), " Averaging went wrong.")
    
    ax[0].errorbar(data[0], data[1], xerr=data_unc[0], yerr=data_unc[1], ls="", color=color, label=label, capsize=2, lw=1)
    
    
fig, ax = create_fig()

draw_scatter(run[runID], fig, ax, 12,3, mask=None, color="black", label="avg. 1k meas. each, \nunc. ~ std. of 1000 meas.", averageN=1000)

Hist.finalize_fig(run[runID], fig, ax[0], xlabel="time [ns]", ylabel="baseline [mV]", title="Baseline vs. Time", logy = False)
fig.savefig("output/Baseline_vs_Time.pdf")